In [1]:
from Base.Century_new.env import*

import numpy as np
import numba as nb


from scipy.stats.mstats import gmean, hmean
from numba import njit

In [2]:
env_state = initEnv()
state = getAgentState(env_state)


In [3]:
@njit()
def bot_lv02(state, perData):
    validActions = getValidActions(state)
    arr_action = np.where(validActions == 1)[0]
    idx = np.random.randint(0, arr_action.shape[0])
    # print('qua')
    perData.append(state)
    return arr_action[idx], perData

In [ ]:
per = []
per.append(np.zeros(P_LENGTH))
win, per = numba_main_2(bot_lv02, 1, per, 0)

In [9]:
len(per), win

(147, 0)

In [2]:
@njit()
def getPredictState(state_original, action):
    state = state_original.copy()
    phase_env = -1
    arr_all_state_predict = np.zeros((1, P_LENGTH))

    phase_env = (
        np.where(state[P_PHASE : P_PHASE + NUMBER_PHASE] == 1)[0][0] + 1
    )
  
    id_action_to_see = np.where(state[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER])[0][0]
    player_in4 = state[:P_ATTRIBUTE_PLAYER].copy()
    # print('action', action)
    if phase_env == 1:
        if action == 0:
            state_predict = state.copy()
            state_predict[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            
            card_hand_player = state_predict[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER]
            card_up_player = state_predict[P_ACTION_CARD_UP_PLAYER : P_ACTION_CARD_UP_PLAYER + NUMBER_ACTION_CARD]
            card_hand_player = np.where(card_up_player == 1, 1, card_hand_player)
            state_predict[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER] = card_hand_player
            state_predict[P_ACTION_CARD_UP_PLAYER : P_ACTION_CARD_UP_PLAYER + NUMBER_ACTION_CARD] = 0
            #chuyển người chơi, về phase 1
            new_phase = 1
            state_predict[P_PHASE + new_phase - 1] = 1
            state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            state_predict[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_predict[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_predict[:P_ATTRIBUTE_PLAYER]))
            
            state_predict = np.reshape(state_predict, (1, P_LENGTH))
            arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
        
        elif action in range(1, 7):
            if action == 1:     #nếu mua thẻ đầu
                state_pre_action1 = state.copy()
                state_pre_action1[P_PHASE : P_PHASE + NUMBER_PHASE] = 0

                top_6_card = state_pre_action1[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD]
                list_action_card_down = state_pre_action1[P_DOWN_ACTION_CARD : P_ID_ACTION]
                id_card_buy = action - 1
                card_buy_in4 = top_6_card[LENGTH_ACTION_CARD * id_card_buy : LENGTH_ACTION_CARD * (id_card_buy + 1)]
                list_card_player = state_pre_action1[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER]
                
                all_token_free = state_pre_action1[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD]
                token_free = all_token_free[NUMBBER_TYPE_TOKEN * id_card_buy : NUMBBER_TYPE_TOKEN * (id_card_buy + 1)]
                all_token_free = np.concatenate((all_token_free[NUMBBER_TYPE_TOKEN:], np.zeros(4)))
                
                card_buy = 100
                for i in range(len(ALL_CARD_IN4)):
                    if np.sum(ALL_CARD_IN4[i] != card_buy_in4) == 0:
                        card_buy = int(i)
                        break
                list_card_player[card_buy] = 1
                #cập nhật thẻ người chơi, tài nguyên người chơi, token free bàn chơi
                state_pre_action1[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER] = list_card_player
                state_pre_action1[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] += token_free
                state_pre_action1[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD] = all_token_free
                # kiểm tra có phải trả tài nguyên ko
                if np.sum(state_pre_action1[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                    new_phase = 4
                    state_pre_action1[P_PHASE + new_phase - 1] = 1
                    state_pre_action1[P_TOKEN_NEED_DROP] = np.sum(state_pre_action1[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
                else: #chuyển người chơi
                    new_phase = 1
                    state_pre_action1[P_PHASE + new_phase - 1] = 1
                    state_pre_action1[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                    new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                    state_pre_action1[P_ID_ACTION + new_id_action_to_see] = 1
                    state_pre_action1[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_action1[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_action1[:P_ATTRIBUTE_PLAYER]))
                #cập nhật các tình huống có thể với 6 thẻ trên bàn
                id_action_card_down = np.where(list_action_card_down)[0]
                if len(id_action_card_down) == 0:
                    state_predict = state_pre_action1.copy()
                    top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[-1]))
                    state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                    
                    state_predict = np.reshape(state_predict, (1, P_LENGTH))
                    arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
                else:
                    for id_action_card in id_action_card_down:
                        state_predict = state_pre_action1.copy()
                        top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[id_action_card]))
                        state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                        state_predict[P_DOWN_ACTION_CARD + id_action_card] = 0
                        state_predict = np.reshape(state_predict, (1, P_LENGTH))
                        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

            else:   #nếu mua thẻ cần đặt token
                #lưu lại thẻ mua, số token cần đặt, chuyển phase
                state_predict = state.copy()
                state_predict[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                id_card_buy = action - 1
                state_predict[P_TOKEN_NEED_DROP] = id_card_buy
                state_predict[P_ID_CARD_BUY + id_card_buy] = 1
                new_phase = 2
                state_predict[P_PHASE + new_phase - 1] = 1
                state_predict = np.reshape(state_predict, (1, P_LENGTH))
                arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

        elif action in range(7, 12):
            state_pre_action7 = state.copy()
            state_pre_action7[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            # state_pre_action7[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            list_point_card_down = state_pre_action7[P_DOWN_POINT_CARD : P_DOWN_ACTION_CARD]
            top_5_card_point = state_pre_action7[P_OPEN_POINT_CARD : P_OPEN_POINT_CARD + NUMBER_OPEN_POINT_CARD * ATTRIBUTE_POINT_CARD]
            id_card_buy = action - 7
            card_buy_in4 = top_5_card_point[LENGTH_POINT_CARD * id_card_buy : LENGTH_POINT_CARD * (id_card_buy + 1)]
            card_buy = 100
            for i in range(len(ALL_CARD_POINT_IN4)):
                if np.sum(ALL_CARD_POINT_IN4[i] != card_buy_in4) == 0:
                    card_buy = i
                    break
            token_fee = card_buy_in4[:4]
            free_score = 0
            if id_card_buy < 2:
                if id_card_buy == 0:
                    if state_pre_action7[P_GOLD_COIN] != 0:
                        free_score = FREE_SCORE_GOLD
                        state_pre_action7[P_GOLD_COIN] -= 1
                    else:
                        if state_pre_action7[P_SILVER_COIN] != 0 and state_pre_action7[P_GOLD_COIN] == 0:
                            free_score = FREE_SCORE_SILVER
                            state_pre_action7[P_SILVER_COIN] -= 1
                else:
                    if state_pre_action7[P_SILVER_COIN] != 0 and state_pre_action7[P_GOLD_COIN] != 0:
                        free_score = FREE_SCORE_SILVER
                        state_pre_action7[P_SILVER_COIN] -= 1
            #cập nhật điểm và tài nguyên và trạng thái game
            state_pre_action7[P_SCORE] += free_score + card_buy_in4[-1]
            state_pre_action7[P_PLAYER_NUMBER_POINT_CARD] += 1
            state_pre_action7[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] -= token_fee    
            if state_pre_action7[P_PLAYER_NUMBER_POINT_CARD] == 5:
                state_pre_action7[P_CHECK_END]  = 1              
            #cập nhật phase mới và chuyển người chơi
            new_phase = 1
            state_pre_action7[P_PHASE + new_phase - 1] = 1
            state_pre_action7[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_pre_action7[P_ID_ACTION + new_id_action_to_see] = 1
            state_pre_action7[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_action7[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_action7[:P_ATTRIBUTE_PLAYER]))
            #cập nhật thẻ điểm trên bàn
            id_point_card_down = np.where(list_point_card_down)[0]
            for id_point_card in id_point_card_down:
                state_predict = state_pre_action7.copy()
                top_5_card_point = np.concatenate((top_5_card_point[ : LENGTH_POINT_CARD * id_card_buy], top_5_card_point[LENGTH_POINT_CARD * (id_card_buy + 1) : ], ALL_CARD_POINT_IN4[id_point_card]))
                state_predict[P_OPEN_POINT_CARD : P_OPEN_POINT_CARD + NUMBER_OPEN_POINT_CARD * ATTRIBUTE_POINT_CARD] = top_5_card_point
                state_predict[P_DOWN_POINT_CARD + id_point_card] = 0

                state_predict = np.reshape(state_predict, (1, P_LENGTH))
                arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

        elif action in range(12, 57):
            #lấy thông tin
            state_pre_action12 = state.copy()
            state_pre_action12[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            # state_pre_action12[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            card_hand_player = state_pre_action12[P_ACTION_CARD_DOWN_PLAYER : P_OTHER_PLAYER_IN4]
            id_card_use = action - 12
            token_fee_get = ALL_CARD_IN4[id_card_use][:ATTRIBUTE_ACTION_CARD]
            card_hand_player[id_card_use] -= 1
            card_hand_player[id_card_use + NUMBER_ACTION_CARD] += 1
            #cập nhật thẻ hành động của người chơi
            state_pre_action12[P_ACTION_CARD_DOWN_PLAYER : P_OTHER_PLAYER_IN4] = card_hand_player
            
            if np.sum(token_fee_get) == 0:  # nếu là thẻ nâng cấp
                state_predict = state_pre_action12.copy()
                state_predict[P_NUMBER_UPGRADE] = ALL_CARD_IN4[id_card_use][-1]
                new_phase = 5
                state_predict[P_PHASE + new_phase - 1] = 1
            else:
                state_predict = state_pre_action12.copy()
                #cập nhật token
                state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] = state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] - token_fee_get[:4] + token_fee_get[4:]
                #nếu thẻ được dùng nhiều lần
                if np.sum(token_fee_get[:4]) > 0 and np.sum(token_fee_get[:4] > state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) == 0:
                    state_predict[P_LAST_ACTION + id_card_use] = 1
                    new_phase = 3
                    state_predict[P_PHASE + new_phase - 1] = 1
                
                else:#thẻ dùng 1 lần rồi bỏ
                    #xem có phải đi trả tài nguyên ko
                    if np.sum(state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) > 10:
                        state_predict[P_TOKEN_NEED_DROP] = np.sum(state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) - 10
                        new_phase = 4
                        state_predict[P_PHASE + new_phase - 1] = 1
                    
                    else:#về phase 1 chuyển người chơi
                        new_phase = 1
                        state_predict[P_PHASE + new_phase - 1] = 1
                        state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                        new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                        state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            
            state_predict = np.reshape(state_predict, (1, P_LENGTH))
            arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
    
    elif phase_env == 2:
        state_pre_phase2 = state.copy()
        state_pre_phase2[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
        stay_drop = int(state_pre_phase2[P_TOKEN_NEED_DROP]) - 1
        all_token_free = state_pre_phase2[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD]
        token_drop = action - 57
        #cập nhật thông tin
        state_pre_phase2[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][token_drop] -= 1
        all_token_free[4 * stay_drop + token_drop] += 1
        state_pre_phase2[P_TOKEN_NEED_DROP] -= 1

        if state_pre_phase2[P_TOKEN_NEED_DROP] == 0: #hoàn tất đặt nguyên liệu thì lấy thẻ
            top_6_card = state_pre_phase2[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD]
            list_action_card_down = state_pre_phase2[P_DOWN_ACTION_CARD : P_ID_ACTION]
            id_card_buy = np.where(state_pre_phase2[P_ID_CARD_BUY : P_ID_CARD_BUY + NUMBER_OPEN_ACTION_CARD])[0][0]
            card_buy_in4 = top_6_card[LENGTH_ACTION_CARD * id_card_buy : LENGTH_ACTION_CARD * (id_card_buy + 1)]
            list_card_player = state_pre_phase2[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER]
            token_free = np.zeros(NUMBBER_TYPE_TOKEN)
            if id_card_buy != 5: #cập nhật giảm token free
                token_free = all_token_free[NUMBBER_TYPE_TOKEN*id_card_buy : NUMBBER_TYPE_TOKEN*(id_card_buy + 1)]
                all_token_free = np.concatenate((all_token_free[: NUMBBER_TYPE_TOKEN * id_card_buy], all_token_free[NUMBBER_TYPE_TOKEN * (id_card_buy + 1):], np.zeros(NUMBBER_TYPE_TOKEN)))
            card_buy = 100
            for i in range(len(ALL_CARD_IN4)):
                if np.sum(ALL_CARD_IN4[i] != card_buy_in4) == 0:
                    card_buy = i
                    break
            list_card_player[card_buy] = 1
            #cập nhật thẻ người chơi, tài nguyên người chơi, token free bàn chơi
            state_pre_phase2[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER] = list_card_player
            state_pre_phase2[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] += token_free
            state_pre_phase2[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD] = all_token_free
            # kiểm tra có phải trả tài nguyên ko
            if np.sum(state_pre_phase2[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                new_phase = 4
                state_pre_phase2[P_PHASE + new_phase - 1] = 1
                state_pre_phase2[P_TOKEN_NEED_DROP] = np.sum(state_pre_phase2[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
            else: #chuyển người chơi
                new_phase = 1
                state_pre_phase2[P_PHASE + new_phase - 1] = 1
                state_pre_phase2[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                state_pre_phase2[P_ID_ACTION + new_id_action_to_see] = 1
                state_pre_phase2[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_phase2[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_phase2[:P_ATTRIBUTE_PLAYER]))
            #cập nhật các tình huống có thể với 6 thẻ trên bàn
            id_action_card_down = np.where(list_action_card_down)[0]
            if len(id_action_card_down) == 0:
                state_predict = state_pre_phase2.copy()

                top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[-1]))
                state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                
                state_predict = np.reshape(state_predict, (1, P_LENGTH))
                arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

            else:
                for id_action_card in id_action_card_down:
                    state_predict = state_pre_phase2.copy()
                    top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[id_action_card]))
                    state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                    state_predict[P_DOWN_ACTION_CARD + id_action_card] = 0

                    state_predict = np.reshape(state_predict, (1, P_LENGTH))
                    arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
        else:
            state_pre_phase2[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD] = all_token_free
            new_phase = 2
            state_pre_phase2[P_PHASE + new_phase - 1] = 1
            state_predict = state_pre_phase2.copy()

            state_predict = np.reshape(state_predict, (1, P_LENGTH))
            arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    elif phase_env == 3:
        state_pre_phase3 = state.copy()
        if action == 61: #nếu ko action tiếp
            state_pre_phase3[P_LAST_ACTION : P_LAST_ACTION + NUMBER_ACTION_CARD] = 0
            #nếu thừa nguyên liệu thì đi lược bỏ
            if np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                new_phase = 4
                state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                state_pre_phase3[P_PHASE + new_phase - 1] = 1
                state_pre_phase3[P_TOKEN_NEED_DROP] = np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
            else: #chuyển người chơi
                new_phase = 1
                state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                state_pre_phase3[P_PHASE + new_phase - 1] = 1
                state_pre_phase3[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                state_pre_phase3[P_ID_ACTION + new_id_action_to_see] = 1
                state_pre_phase3[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_phase3[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_phase3[:P_ATTRIBUTE_PLAYER]))
        else:# Lấy thông tin
            id_card_use = np.where(state_pre_phase3[P_LAST_ACTION : P_LAST_ACTION + NUMBER_ACTION_CARD])[0][0]
            token_fee_get = ALL_CARD_IN4[id_card_use][:ATTRIBUTE_ACTION_CARD]
            #cập nhật thông tin
            state_pre_phase3[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] = state_pre_phase3[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] + token_fee_get[NUMBBER_TYPE_TOKEN : ] - token_fee_get[: NUMBBER_TYPE_TOKEN]
            #kiểm tra xem hành động tiếp được ko
            if np.sum(token_fee_get[:NUMBBER_TYPE_TOKEN] > state_pre_phase3[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) > 0:
                state_pre_phase3[P_LAST_ACTION : P_LAST_ACTION + NUMBER_ACTION_CARD] = 0
                #nếu thừa nguyên liệu thì đi lược bỏ
                if np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                    new_phase = 4
                    state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                    state_pre_phase3[P_PHASE + new_phase - 1] = 1
                    state_pre_phase3[P_TOKEN_NEED_DROP] = np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
                else: #chuyển người chơi
                    new_phase = 1
                    state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                    state_pre_phase3[P_PHASE + new_phase - 1] = 1
                    state_pre_phase3[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                    new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                    state_pre_phase3[P_ID_ACTION + new_id_action_to_see] = 1
                    state_pre_phase3[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_phase3[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_phase3[:P_ATTRIBUTE_PLAYER]))

        state_predict = state_pre_phase3.copy()
        state_predict = np.reshape(state_predict, (1, P_LENGTH))
        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    elif phase_env == 4:
        state_predict = state.copy()
        token_drop = action - 57
        state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][token_drop] -= 1
        state_predict[P_TOKEN_NEED_DROP] -= 1
        if state_predict[P_TOKEN_NEED_DROP] == 0:
            state_predict[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            new_phase = 1
            state_predict[P_PHASE + new_phase - 1] = 1
            state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            state_predict[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_predict[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_predict[:P_ATTRIBUTE_PLAYER]))
        
        state_predict = np.reshape(state_predict, (1, P_LENGTH))
        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    elif phase_env == 5:
        state_predict = state.copy()
        id_update = action - 62 
        state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][id_update] -= 1
        state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][id_update + 1] += 1
        state_predict[P_NUMBER_UPGRADE] -= 1

        if state_predict[P_NUMBER_UPGRADE] == 0 or np.sum(state_predict[P_TOKEN : P_TOKEN + NUMBBER_TYPE_TOKEN - 1]) == 0:
            state_predict[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            new_phase = 1
            state_predict[P_PHASE + new_phase - 1] = 1
            state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            state_predict[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_predict[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_predict[:P_ATTRIBUTE_PLAYER]))
        
        state_predict = np.reshape(state_predict, (1, P_LENGTH))
        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    arr_all_predict_state, arr_prob = get_Prob_State(arr_all_state_predict[1:])

    return arr_all_predict_state, arr_prob


In [3]:
@njit()
def bot_monte(state, perData):
    validActions = getValidActions(state)
    arr_action = np.where(validActions == 1)[0]
    action_return = -1
    count_min = 10000
    
    # print('RUN lượt 1')
    for action in arr_action:
        state_test = state.copy()
        action_test = action
        # print('CHECK', arr_action, getReward(state_test))
        id_action = 0
        count_change = 0
        count = 0
        while getReward(state_test) == -1:
            try:
                arr_next_state, arr_prob_next = getPredictState(state_test, action_test)
                # print(ACTIONS_MEAN[action_test], state_test[P_ID_ACTION:P_ID_ACTION + NUMBER_PLAYER], state_test[1:500:96])
                state_test_i = arr_next_state[np.random.randint(0, len(arr_prob_next))]       #chọn ngẫu nhiên 1 state
            except:
                print(list(state_test))
                raise Exception('toang predict')
            
            state_test = arr_next_state[np.random.randint(0, len(arr_prob_next))]       #chọn ngẫu nhiên 1 state
            # state_test = arr_next_state[0]
            id_real_action = np.where(state_test[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER])[0][0]
            
            if id_real_action != id_action:
                count_change += 1
                id_action = id_real_action

            arr_action_test = np.where(getValidActions(state_test))[0]                  #các action khả dĩ ở state được chọn
            action_test = np.random.choice(arr_action_test)                             #chọn ngẫu nhiên 1 action

        count_turn = count_change // 4

        if count_turn < count_min:
            count_min = count_turn
            action_return = action
    # print('check', action_return, count_min, ACTIONS_MEAN[action_return])

    return action_return, perData

per = np.zeros(10)

win, per = numba_main_2(bot_monte, 10, per, 0)
print('DONE', win)

DONE 5


In [4]:
per = np.zeros(10)

win, per = numba_main_2(bot_monte, 10, per, 0)
print('DONE', win)

DONE 4


In [15]:
state = np.array([0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 80.0, 0.0, 0.0, 4.0, 0.0, 120.0, 1.0, 1.0, 1.0, 3.0, 200.0, 1.0, 1.0, 1.0, 1.0, 120.0, 2.0, 0.0, 0.0, 2.0, 100.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 3.0, 4.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
)

state[P_ID_ACTION:P_ID_ACTION + NUMBER_PLAYER]

array([1., 0., 0., 0., 0.])

In [17]:
state[P_PHASE : P_PHASE+NUMBER_PHASE]

array([0., 1., 0., 0., 0.])

In [18]:
np.where(state[P_ID_CARD_BUY : P_ID_CARD_BUY + NUMBER_OPEN_ACTION_CARD])[0][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [16]:
state[P_TOKEN_NEED_DROP]
arr_action = np.where(getValidActions(state))
arr_action

(array([57], dtype=int64),)

In [6]:
getPredictState(state, 57)

(array([[0., 0., 3., ..., 0., 0., 0.],
        [0., 0., 3., ..., 0., 0., 0.],
        [0., 0., 3., ..., 0., 0., 0.],
        ...,
        [0., 0., 3., ..., 0., 0., 0.],
        [0., 0., 3., ..., 0., 0., 0.],
        [0., 0., 3., ..., 0., 0., 0.]]),
 array([0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778]))

In [12]:


@njit()
def getPredictState(state_original, action):
    state = state_original.copy()
    phase_env = -1
    arr_all_state_predict = np.zeros((1, P_LENGTH))

    phase_env = (
        np.where(state[P_PHASE : P_PHASE + NUMBER_PHASE] == 1)[0][0] + 1
    )
  
    id_action_to_see = np.where(state[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER])[0][0]
    # player_in4 = state[:P_ATTRIBUTE_PLAYER].copy()
    print('action', action)
    if phase_env == 1:
        if action == 0:
            state_predict = state.copy()
            state_predict[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            
            card_hand_player = state_predict[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER]
            card_up_player = state_predict[P_ACTION_CARD_UP_PLAYER : P_ACTION_CARD_UP_PLAYER + NUMBER_ACTION_CARD]
            card_hand_player = np.where(card_up_player == 1, 1, card_hand_player)
            state_predict[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER] = card_hand_player
            state_predict[P_ACTION_CARD_UP_PLAYER : P_ACTION_CARD_UP_PLAYER + NUMBER_ACTION_CARD] = 0
            #chuyển người chơi, về phase 1
            new_phase = 1
            state_predict[P_PHASE + new_phase - 1] = 1
            state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            state_predict[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_predict[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_predict[:P_ATTRIBUTE_PLAYER]))
            
            state_predict = np.reshape(state_predict, (1, P_LENGTH))
            arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
        
        elif action in range(1, 7):
            if action == 1:     #nếu mua thẻ đầu
                state_pre_action1 = state.copy()
                state_pre_action1[P_PHASE : P_PHASE + NUMBER_PHASE] = 0

                top_6_card = state_pre_action1[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD]
                list_action_card_down = state_pre_action1[P_DOWN_ACTION_CARD : P_ID_ACTION]
                id_card_buy = action - 1
                card_buy_in4 = top_6_card[LENGTH_ACTION_CARD * id_card_buy : LENGTH_ACTION_CARD * (id_card_buy + 1)]
                list_card_player = state_pre_action1[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER]
                
                all_token_free = state_pre_action1[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD]
                token_free = all_token_free[NUMBBER_TYPE_TOKEN * id_card_buy : NUMBBER_TYPE_TOKEN * (id_card_buy + 1)]
                all_token_free = np.concatenate((all_token_free[NUMBBER_TYPE_TOKEN:], np.zeros(4)))
                
                card_buy = 100
                for i in range(len(ALL_CARD_IN4)):
                    if np.sum(ALL_CARD_IN4[i] != card_buy_in4) == 0:
                        card_buy = i
                        break
                list_card_player[card_buy] = 1
                #cập nhật thẻ người chơi, tài nguyên người chơi, token free bàn chơi
                state_pre_action1[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER] = list_card_player
                state_pre_action1[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] += token_free
                state_pre_action1[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD] = all_token_free
                # kiểm tra có phải trả tài nguyên ko
                if np.sum(state_pre_action1[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                    new_phase = 4
                    state_pre_action1[P_PHASE + new_phase - 1] = 1
                    state_pre_action1[P_TOKEN_NEED_DROP] = np.sum(state_pre_action1[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
                else: #chuyển người chơi
                    new_phase = 1
                    state_pre_action1[P_PHASE + new_phase - 1] = 1
                    state_pre_action1[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                    new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                    state_pre_action1[P_ID_ACTION + new_id_action_to_see] = 1
                    state_pre_action1[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_action1[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_action1[:P_ATTRIBUTE_PLAYER]))
                #cập nhật các tình huống có thể với 6 thẻ trên bàn
                id_action_card_down = np.where(list_action_card_down)[0]
                if len(id_action_card_down) == 0:
                    state_predict = state_pre_action1.copy()
                    top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[-1]))
                    state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                    
                    state_predict = np.reshape(state_predict, (1, P_LENGTH))
                    arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
                else:
                    for id_action_card in id_action_card_down:
                        state_predict = state_pre_action1.copy()
                        top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[id_action_card]))
                        state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                        state_predict[P_DOWN_ACTION_CARD + id_action_card] = 0
                        state_predict = np.reshape(state_predict, (1, P_LENGTH))
                        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

            else:   #nếu mua thẻ cần đặt token
                #lưu lại thẻ mua, số token cần đặt, chuyển phase
                state_pre_action_else = state.copy()
                state_pre_action_else[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                id_card_buy = action - 1
                state_pre_action_else[P_TOKEN_NEED_DROP] = id_card_buy
                state_pre_action_else[P_ID_CARD_BUY + id_card_buy] = 1
                new_phase = 2
                state_pre_action_else[P_PHASE + new_phase - 1] = 1

        elif action in range(7, 12):
            state_pre_action7 = state.copy()
            state_pre_action7[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            # state_pre_action7[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            list_point_card_down = state_pre_action7[P_DOWN_POINT_CARD : P_DOWN_ACTION_CARD]
            top_5_card_point = state_pre_action7[P_OPEN_POINT_CARD : P_OPEN_POINT_CARD + NUMBER_OPEN_POINT_CARD * ATTRIBUTE_POINT_CARD]
            id_card_buy = action - 7
            card_buy_in4 = top_5_card_point[LENGTH_POINT_CARD * id_card_buy : LENGTH_POINT_CARD * (id_card_buy + 1)]
            card_buy = 100
            for i in range(len(ALL_CARD_POINT_IN4)):
                if np.sum(ALL_CARD_POINT_IN4[i] != card_buy_in4) == 0:
                    card_buy = i
                    break
            token_fee = card_buy_in4[:4]
            free_score = 0
            if id_card_buy < 2:
                if id_card_buy == 0:
                    if state_pre_action7[P_GOLD_COIN] != 0:
                        free_score = FREE_SCORE_GOLD
                        state_pre_action7[P_GOLD_COIN] -= 1
                    else:
                        if state_pre_action7[P_SILVER_COIN] != 0 and state_pre_action7[P_GOLD_COIN] == 0:
                            free_score = FREE_SCORE_SILVER
                            state_pre_action7[P_SILVER_COIN] -= 1
                else:
                    if state_pre_action7[P_SILVER_COIN] != 0 and state_pre_action7[P_GOLD_COIN] != 0:
                        free_score = FREE_SCORE_SILVER
                        state_pre_action7[P_SILVER_COIN] -= 1
            #cập nhật điểm và tài nguyên
            state_pre_action7[P_SCORE] += free_score + card_buy_in4[-1]
            state_pre_action7[P_PLAYER_NUMBER_POINT_CARD] += 1
            state_pre_action7[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] -= token_fee                   
            #cập nhật phase mới và chuyển người chơi
            new_phase = 1
            state_pre_action7[P_PHASE + new_phase - 1] = 1
            state_pre_action7[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_pre_action7[P_ID_ACTION + new_id_action_to_see] = 1
            state_pre_action7[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_action7[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_action7[:P_ATTRIBUTE_PLAYER]))
            #cập nhật thẻ điểm trên bàn
            id_point_card_down = np.where(list_point_card_down)[0]
            for id_point_card in id_point_card_down:
                state_predict = state_pre_action7.copy()
                top_5_card_point = np.concatenate((top_5_card_point[ : LENGTH_POINT_CARD * id_card_buy], top_5_card_point[LENGTH_POINT_CARD * (id_card_buy + 1) : ], ALL_CARD_POINT_IN4[id_point_card]))
                state_predict[P_OPEN_POINT_CARD : P_OPEN_POINT_CARD + NUMBER_OPEN_POINT_CARD * ATTRIBUTE_POINT_CARD] = top_5_card_point
                state_predict[P_DOWN_POINT_CARD + id_point_card] = 0

                state_predict = np.reshape(state_predict, (1, P_LENGTH))
                arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

        elif action in range(12, 57):
            #lấy thông tin
            state_pre_action12 = state.copy()
            state_pre_action12[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            # state_pre_action12[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            card_hand_player = state_pre_action12[P_ACTION_CARD_DOWN_PLAYER : P_OTHER_PLAYER_IN4]
            id_card_use = action - 12
            token_fee_get = ALL_CARD_IN4[id_card_use][:ATTRIBUTE_ACTION_CARD]
            card_hand_player[id_card_use] -= 1
            card_hand_player[id_card_use + NUMBER_ACTION_CARD] += 1
            #cập nhật thẻ hành động của người chơi
            state_pre_action12[P_ACTION_CARD_DOWN_PLAYER : P_OTHER_PLAYER_IN4] = card_hand_player
            
            if np.sum(token_fee_get) == 0:  # nếu là thẻ nâng cấp
                state_predict = state_pre_action12.copy()
                state_predict[P_NUMBER_UPGRADE] = ALL_CARD_IN4[id_card_use][-1]
                new_phase = 5
                state_predict[P_PHASE + new_phase - 1] = 1
            else:
                state_predict = state_pre_action12.copy()
                #cập nhật token
                state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] = state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] - token_fee_get[:4] + token_fee_get[4:]
                #nếu thẻ được dùng nhiều lần
                if np.sum(token_fee_get[:4]) > 0 and np.sum(token_fee_get[:4] > state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) == 0:
                    state_predict[P_LAST_ACTION + id_card_use] = 1
                    new_phase = 3
                    state_predict[P_PHASE + new_phase - 1] = 1
                
                else:#thẻ dùng 1 lần rồi bỏ
                    #xem có phải đi trả tài nguyên ko
                    if np.sum(state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) > 10:
                        state_predict[P_TOKEN_NEED_DROP] = np.sum(state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) - 10
                        new_phase = 4
                        state_predict[P_PHASE + new_phase - 1] = 1
                    
                    else:#về phase 1 chuyển người chơi
                        new_phase = 1
                        state_predict[P_PHASE + new_phase - 1] = 1
                        state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                        new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                        state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            
            state_predict = np.reshape(state_predict, (1, P_LENGTH))
            arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
    
    elif phase_env == 2:
        state_pre_phase2 = state.copy()
        state_pre_phase2[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
        stay_drop = int(state_pre_phase2[P_TOKEN_NEED_DROP]) - 1
        all_token_free = state_pre_phase2[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD]
        token_drop = action - 57
        #cập nhật thông tin
        state_pre_phase2[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][token_drop] -= 1
        all_token_free[4 * stay_drop + token_drop] += 1
        state_pre_phase2[P_TOKEN_NEED_DROP] -= 1

        if state_pre_phase2[P_TOKEN_NEED_DROP] == 0: #hoàn tất đặt nguyên liệu thì lấy thẻ
            top_6_card = state_pre_phase2[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD]
            list_action_card_down = state_pre_phase2[P_DOWN_ACTION_CARD : P_ID_ACTION]
            id_card_buy = np.where(state_pre_phase2[P_ID_CARD_BUY : P_ID_CARD_BUY + NUMBER_OPEN_ACTION_CARD])[0][0]
            card_buy_in4 = top_6_card[LENGTH_ACTION_CARD * id_card_buy : LENGTH_ACTION_CARD * (id_card_buy + 1)]
            list_card_player = state_pre_phase2[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER]
            token_free = np.zeros(NUMBBER_TYPE_TOKEN)
            if id_card_buy != 5: #cập nhật giảm token free
                token_free = all_token_free[NUMBBER_TYPE_TOKEN*id_card_buy : NUMBBER_TYPE_TOKEN*(id_card_buy + 1)]
                all_token_free = np.concatenate((all_token_free[: NUMBBER_TYPE_TOKEN * id_card_buy], all_token_free[NUMBBER_TYPE_TOKEN * (id_card_buy + 1):], np.zeros(NUMBBER_TYPE_TOKEN)))
            card_buy = 100
            for i in range(len(ALL_CARD_IN4)):
                if np.sum(ALL_CARD_IN4[i] != card_buy_in4) == 0:
                    card_buy = i
                    break
            list_card_player[card_buy] = 1
            #cập nhật thẻ người chơi, tài nguyên người chơi, token free bàn chơi
            state_pre_phase2[BASIC_ATTRIBUTE_PLAYER : P_ACTION_CARD_UP_PLAYER] = list_card_player
            state_pre_phase2[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] += token_free
            state_pre_phase2[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD] = all_token_free
            # kiểm tra có phải trả tài nguyên ko
            if np.sum(state_pre_phase2[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                new_phase = 4
                state_pre_phase2[P_PHASE + new_phase - 1] = 1
                state_pre_phase2[P_TOKEN_NEED_DROP] = np.sum(state_pre_phase2[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
            else: #chuyển người chơi
                new_phase = 1
                state_pre_phase2[P_PHASE + new_phase - 1] = 1
                state_pre_phase2[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                state_pre_phase2[P_ID_ACTION + new_id_action_to_see] = 1
                state_pre_phase2[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_phase2[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_phase2[:P_ATTRIBUTE_PLAYER]))
            #cập nhật các tình huống có thể với 6 thẻ trên bàn
            id_action_card_down = np.where(list_action_card_down)[0]
            if len(id_action_card_down) == 0:
                state_predict = state_pre_phase2.copy()

                top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[-1]))
                state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                
                state_predict = np.reshape(state_predict, (1, P_LENGTH))
                arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

            else:
                for id_action_card in id_action_card_down:
                    state_predict = state_pre_phase2.copy()
                    top_6_card = np.concatenate((top_6_card[LENGTH_ACTION_CARD : ], ALL_CARD_IN4[id_action_card]))
                    state_predict[P_OPEN_ACTION_CARD : P_TOKEN_ON_ACTION_CARD] = top_6_card
                    state_predict[P_DOWN_ACTION_CARD + id_action_card] = 0

                    state_predict = np.reshape(state_predict, (1, P_LENGTH))
                    arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)
        else:
            state_pre_phase2[P_TOKEN_ON_ACTION_CARD : P_OPEN_POINT_CARD] = all_token_free
            new_phase = 2
            state_pre_phase2[P_PHASE + new_phase - 1] = 1
            state_predict = state_pre_phase2.copy()

            state_predict = np.reshape(state_predict, (1, P_LENGTH))
            arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    elif phase_env == 3:
        state_pre_phase3 = state.copy()
        if action == 61: #nếu ko action tiếp
            state_pre_phase3[P_LAST_ACTION : P_LAST_ACTION + NUMBER_ACTION_CARD] = 0
            #nếu thừa nguyên liệu thì đi lược bỏ
            if np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                new_phase = 4
                state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                state_pre_phase3[P_PHASE + new_phase - 1] = 1
                state_pre_phase3[P_TOKEN_NEED_DROP] = np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
            else: #chuyển người chơi
                new_phase = 1
                state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                state_pre_phase3[P_PHASE + new_phase - 1] = 1
                state_pre_phase3[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                state_pre_phase3[P_ID_ACTION + new_id_action_to_see] = 1
                state_pre_phase3[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_phase3[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_phase3[:P_ATTRIBUTE_PLAYER]))
        else:# Lấy thông tin
            id_card_use = np.where(state_pre_phase3[P_LAST_ACTION : P_LAST_ACTION + NUMBER_ACTION_CARD])[0][0]
            token_fee_get = ALL_CARD_IN4[id_card_use][:ATTRIBUTE_ACTION_CARD]
            #cập nhật thông tin
            state_pre_phase3[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] = state_pre_phase3[P_TOKEN : BASIC_ATTRIBUTE_PLAYER] + token_fee_get[NUMBBER_TYPE_TOKEN : ] - token_fee_get[: NUMBBER_TYPE_TOKEN]
            #kiểm tra xem hành động tiếp được ko
            if np.sum(token_fee_get[:NUMBBER_TYPE_TOKEN] > state_pre_phase3[P_TOKEN : BASIC_ATTRIBUTE_PLAYER]) > 0:
                state_pre_phase3[P_LAST_ACTION : P_LAST_ACTION + NUMBER_ACTION_CARD] = 0
                #nếu thừa nguyên liệu thì đi lược bỏ
                if np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) > 10:  #thừa token thì phải đi bỏ, ko cần chuyển người chơi
                    new_phase = 4
                    state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                    state_pre_phase3[P_PHASE + new_phase - 1] = 1
                    state_pre_phase3[P_TOKEN_NEED_DROP] = np.sum(state_pre_phase3[P_TOKEN:BASIC_ATTRIBUTE_PLAYER]) - 10
                else: #chuyển người chơi
                    new_phase = 1
                    state_pre_phase3[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
                    state_pre_phase3[P_PHASE + new_phase - 1] = 1
                    state_pre_phase3[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
                    new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
                    state_pre_phase3[P_ID_ACTION + new_id_action_to_see] = 1
                    state_pre_phase3[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_pre_phase3[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_pre_phase3[:P_ATTRIBUTE_PLAYER]))

        state_predict = state_pre_phase3.copy()
        state_predict = np.reshape(state_predict, (1, P_LENGTH))
        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    elif phase_env == 4:
        state_predict = state.copy()
        token_drop = action - 57
        state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][token_drop] -= 1
        state_predict[P_TOKEN_NEED_DROP] -= 1
        if state_predict[P_TOKEN_NEED_DROP] == 0:
            state_predict[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            new_phase = 1
            state_predict[P_PHASE + new_phase - 1] = 1
            state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            state_predict[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_predict[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_predict[:P_ATTRIBUTE_PLAYER]))
        
        state_predict = np.reshape(state_predict, (1, P_LENGTH))
        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    elif phase_env == 5:
        state_predict = state.copy()
        id_update = action - 62 
        state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][id_update] -= 1
        state_predict[P_TOKEN : BASIC_ATTRIBUTE_PLAYER][id_update + 1] += 1
        state_predict[P_NUMBER_UPGRADE] -= 1

        if state_predict[P_NUMBER_UPGRADE] == 0 or np.sum(state_predict[P_TOKEN : P_TOKEN + NUMBBER_TYPE_TOKEN - 1]) == 0:
            state_predict[P_PHASE : P_PHASE + NUMBER_PHASE] = 0
            new_phase = 1
            state_predict[P_PHASE + new_phase - 1] = 1
            state_predict[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER] = 0
            new_id_action_to_see = (id_action_to_see + 1) % NUMBER_PLAYER
            state_predict[P_ID_ACTION + new_id_action_to_see] = 1
            state_predict[: P_ATTRIBUTE_PLAYER * NUMBER_PLAYER] = np.concatenate((state_predict[P_ATTRIBUTE_PLAYER : P_ATTRIBUTE_PLAYER*NUMBER_PLAYER], state_predict[:P_ATTRIBUTE_PLAYER]))
        
        state_predict = np.reshape(state_predict, (1, P_LENGTH))
        arr_all_state_predict = np.append(arr_all_state_predict, state_predict, axis= 0)

    arr_all_predict_state, arr_prob = get_Prob_State(arr_all_state_predict[1:])

    return arr_all_predict_state, arr_prob
    # return arr_all_state_predict





In [13]:
state = per[1]
arr_action = np.where(getValidActions(state))
print(arr_action)

(array([ 0,  1,  2,  3,  4,  5, 12, 55], dtype=int64),)


In [16]:
getPredictState(state, 1)

action 1


(array([[0., 0., 4., ..., 0., 0., 0.],
        [0., 0., 4., ..., 0., 0., 0.],
        [0., 0., 4., ..., 0., 0., 0.],
        ...,
        [0., 0., 4., ..., 0., 0., 0.],
        [0., 0., 4., ..., 0., 0., 0.],
        [0., 0., 4., ..., 0., 0., 0.]]),
 array([0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778]))

In [11]:
print('done')

done
